In [3]:
# use sqlalchemy to write a orm query to a sqlite database

In [4]:
from sqlalchemy import create_engine, text
import pandas as pd

In [5]:
# Create a SQLite engine
engine = create_engine('sqlite:///../../data/AllPrintings.sqlite')

# Test the connection
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1"))
        print("Connection successful:", result.scalar() == 1)
except Exception as e:
    print("Connection failed:", str(e))


Connection successful: True


Here we'll use the 'text' approach

In [6]:
# Define the SQL command to create the view
drop_view = """
DROP VIEW IF EXISTS standard_cards_view;
"""

create_view_sql = """
CREATE VIEW standard_cards_view AS
SELECT
    cl.standard, 
    c.name, 
    c.setCode, 
    cs.releaseDate,
    c.number, 
    c.layout, 
    c.*
FROM cards AS c
JOIN cardLegalities AS cl ON cl.uuid = c.uuid
JOIN cardPurchaseUrls AS cp ON cp.uuid = c.uuid
JOIN sets AS cs ON cs.code = c.setCode
WHERE cl.standard = 'Legal' 
    AND c.setCode = 'OTJ'
    AND c.isPromo IS NULL 
    AND c.borderColor = 'black' 
    AND c.isReprint IS NULL 
    AND c.promoTypes IS NULL 
ORDER BY c.name ASC, cs.releaseDate ASC, c.power DESC;
"""

# Execute the SQL command
with engine.connect() as connection:
    connection.execute(text(drop_view))
    connection.execute(text(create_view_sql))

print("View 'standard_cards_view' created successfully.")

View 'standard_cards_view' created successfully.


In [12]:
# Define the SQL command to create the view
drop_view = """
DROP VIEW IF EXISTS OTJ_set_view;
"""

create_view_sql = """
CREATE VIEW OTJ_set_view AS
SELECT
    cl.standard, 
    c.name, 
    c.setCode, 
    cs.releaseDate,
    c.number, 
    c.layout, 
    c.*
FROM cards AS c
JOIN cardLegalities AS cl ON cl.uuid = c.uuid
JOIN cardPurchaseUrls AS cp ON cp.uuid = c.uuid
JOIN sets AS cs ON cs.code = c.setCode
WHERE c.setCode IN ('OTJ', 'BIG', 'OTP')
ORDER BY c.name ASC, cs.releaseDate ASC, c.power DESC;
"""

# Execute the SQL command
with engine.connect() as connection:
    connection.execute(text(drop_view))
    connection.execute(text(create_view_sql))

print("View 'OTJ_set_view' created successfully.")

View 'OTJ_set_view' created successfully.


In [16]:

# Define the SQL query
sql_query = """
WITH ranked_cards AS (
    SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY name ORDER BY releaseDate ASC, power DESC) AS rn
    FROM OTJ_set_view
)
SELECT
    name, 
    setCode, 
    releaseDate,
    number, 
    layout,
    availability,
    power, toughness,
    colorIdentity, colors,
    types, subtypes, supertypes,
    manaCost, manaValue,
    edhrecRank, edhrecSaltiness
    -- text, flavorText
FROM ranked_cards
WHERE rn = 1
ORDER BY name ASC;
"""

# Execute the query and load the results into a DataFrame
with engine.connect() as connection:
    df = pd.read_sql_query(sql_query, connection)

# Display the DataFrame
df.to_feather('../../data/OTJ/card.feather')
print(df.shape)
df.head()

(371, 17)


,name,setCode,releaseDate,number,layout,availability,power,toughness,colorIdentity,colors,types,subtypes,supertypes,manaCost,manaValue,edhrecRank,edhrecSaltiness
0,Abraded Bluffs,OTJ,2024-04-19,251,normal,"arena, mtgo, paper",None,None,"R, W",,Land,Desert,,None,0.0,5898.0,NaN
1,Abrupt Decay,OTP,2024-04-19,34,normal,"arena, mtgo, paper",None,None,"B, G","B, G",Instant,,,{B}{G},2.0,625.0,0.31
2,Akul the Unrepentant,OTJ,2024-04-19,346,normal,"arena, mtgo, paper",5,5,"B, R","B, R",Creature,"Scorpion, Dragon, Rogue",Legendary,{B}{B}{R}{R},4.0,12398.0,NaN
3,Aloe Alchemist,OTJ,2024-04-19,152,normal,"arena, mtgo, paper",3,2,G,G,Creature,"Plant, Warlock",,{1}{G},2.0,17373.0,NaN
4,Ambush Gigapede,OTJ,2024-04-19,77,normal,"arena, mtgo, paper",6,2,B,B,Creature,Insect,,{4}{B}{B},6.0,21172.0,NaN
